# **1. Problema**

Identificação de padrão de inadimplência sob base de dados de dados de conseção de crédito. *Default* = 1 (inadimplentes) alvo da análise.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# **2. Carregameneto de dados:**

In [ ]:
URL = "../input/public/credito.csv"
df = pd.read_csv(URL, na_values='na')

df.head()



# **3. Exploração de dados:**

## *3.1 Massa bruta de dados*

**Métricas gerais da massa de dados para análise.**

In [ ]:
linhas, colunas = df.shape
print(f'A matriz de dados tem {linhas} linhas e {colunas} colunas.')

In [ ]:
qtd_total, _ = df.shape
qtd_adimplentes, _ = df[df['default'] == 0].shape
qtd_inadimplentes, _ = df[df['default'] == 1].shape

print(f"A proporcão clientes adimplentes é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"A proporcão clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")

In [ ]:
df.select_dtypes('object').describe().transpose() # limite_credito e valor_transacoes_12m indevidamente classificados como string

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose() # métricas dos valores númericos (faltando os dois campos citados anteriormente)

## *3.2 Tratando dados faltantes*

**Verificando o montante de dados faltantes que possam comprometer a leitura adequeda da amostra de dados.**

In [ ]:
df.isna().any() # identificação das colunas com dados faltantes

In [ ]:
def stats_dados_faltantes(df: pd.DataFrame) -> None:

  stats_dados_faltantes = []
  for col in df.columns:
    if df[col].isna().any():
      qtd, _ = df[df[col].isna()].shape
      total, _ = df.shape
      dict_dados_faltantes = {col: {'quantidade': qtd, "porcentagem": round(100 * qtd/total, 2)}}
      stats_dados_faltantes.append(dict_dados_faltantes)

  for stat in stats_dados_faltantes:
    print(stat)
  
print('\033[1m' + 'Dados faltantes base completa:' + '\033[0m')
stats_dados_faltantes(df=df) # mensuração da quantidade de dados faltantes

In [ ]:
print('\033[1m' + 'Dados faltantes clientes adiplentes:' + '\033[0m')
stats_dados_faltantes(df=df[df['default'] == 0])

print('\n\033[1m' + 'Dados faltantes clientes inadiplentes:' + '\033[0m')
stats_dados_faltantes(df=df[df['default'] == 1])

# **4. Limpenza de dados**

**Conversão de string indevida em float e limpeza de dados vazios**

## *4.1 Correção dos valores*

**Uma vez identificadas as colunas que devem ser tratadas como números(float) fazemos a conversão dos dados.**

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].dtypes # antes da conversão para float


In [ ]:
fn = lambda valor: float(valor.replace(".", "").replace(",", ".")) # teste de função para ajuste dos valores em float

valores_originais = ['12.691,51', '8.256,96', '3.418,56', '3.313,03', '4.716,22']
valores_limpos = list(map(fn, valores_originais))

print(valores_originais)
print(valores_limpos)

In [ ]:
# aplicação do teste na base real
df['valor_transacoes_12m'] = df['valor_transacoes_12m'].apply(lambda valor: float(valor.replace(".", "").replace(",", ".")))
df['limite_credito'] = df['limite_credito'].apply(lambda valor: float(valor.replace(".", "").replace(",", ".")))

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose() # métricas corrigidas após ajuste dos floats

In [ ]:
df.select_dtypes('object').describe().transpose()# métricas corrigidas após ajuste dos floats

## *4.2 Limpeza de dados*

**Remoção das linhas de dados com valores vazios nas colunas.**

In [ ]:
df.dropna(inplace=True)

qtd_total_novo, _ = df.shape
qtd_adimplentes_novo, _ = df[df['default'] == 0].shape
qtd_inadimplentes_novo, _ = df[df['default'] == 1].shape

print(f"A proporcão anterior de adimplentes ativos é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"A nova proporcão de clientes adimplentes é de {round(100 * qtd_adimplentes_novo / qtd_total_novo, 2)}%")
print("")
print(f"A proporcão anterior de clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")
print(f"A nova proporcão de clientes inadimplentes é de {round(100 * qtd_inadimplentes_novo / qtd_total_novo, 2)}%")

# **5. Análise de dados**

## 5.1 Análise por categorias

A primeira análise baseada nas categorias para descobrir possível relação com os eventos alvo do default.
Classes analisadas: *escolaridade, salário anual e estado civil.*

In [ ]:
df.select_dtypes('object').head(n=5)

In [ ]:
sns.set_style("whitegrid")

df_adimplente = df[df['default'] == 0]
df_inadimplente = df[df['default'] == 1]

In [ ]:
def categoria_graficos (coluna: str, titulos: list): # função gera sequência de subplots por categoria

  eixo = 0
  max_y = 0
  max = df.select_dtypes('object').describe()[coluna]['freq'] * 1.1

  figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

  for dataframe in [df, df_adimplente, df_inadimplente]:

    df_to_plot = dataframe[coluna].value_counts().to_frame()
    df_to_plot.rename(columns={coluna: 'frequencia_absoluta'}, inplace=True)
    df_to_plot[coluna] = df_to_plot.index
    df_to_plot.sort_values(by=[coluna], inplace=True)
    df_to_plot.sort_values(by=[coluna])

    f = sns.barplot(x=df_to_plot[coluna], y=df_to_plot['frequencia_absoluta'], ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')
    f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)

    _, max_y_f = f.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y
    f.set(ylim=(0, max_y))

    eixo += 1

  return figura

In [ ]:
colunas = ['escolaridade', 'salario_anual', 'estado_civil']
titulos = ['Total dos Clientes', 'Clientes Adimplentes', 'Clientes Inadimplentes']

figura = plt

for coluna in colunas:
  figura.show(categoria_graficos(coluna, titulos))

## 5.2 Análise numérica

Análise das variáveis númericas da amostra. Variáveis analisadas: *quantidade de transações nos últimos 12 meses e valor das transações dos últimos 12 meses.*

In [ ]:
df.drop(['id', 'default'], axis=1).select_dtypes('number').head(n=5)

In [ ]:
def variaveis_graficos (coluna: str, titulos: list): # função gera sequência de subplots por categoria

  eixo = 0
  max_y = 0
  figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

  for dataframe in [df, df_adimplente, df_inadimplente]:

    f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

    _, max_y_f = f.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y
    f.set(ylim=(0, max_y))

    eixo += 1
  
  return figura


In [ ]:
colunas = ['qtd_transacoes_12m', 'valor_transacoes_12m']
titulos = ['Qtd. de Transações no Último Ano', 'Qtd. de Transações no Último Ano de Adimplentes', 'Qtd. de Transações no Último Ano de Inadimplentes']

for coluna in colunas:
  figura.show(variaveis_graficos(coluna, titulos))

Com base no resultado dos histogramas das duas variáveis estudadas temos abaixo a relação entre ambas no gráfico de relacionamento.

In [ ]:
f = sns.relplot(x='valor_transacoes_12m', y='qtd_transacoes_12m', data=df, hue='default')
_ = f.set(
    title='Relação entre Valor e Quantidade de Transações no Último Ano', 
    xlabel='Valor das Transações no Último Ano', 
    ylabel='Quantidade das Transações no Último Ano'
  )

# **6. Insigths**

## 6.1 Dados descartados

O percentual de dados vazios que foram descartados da amostra não compromete a validade da massa de dados, pois a proporção entre clientes adiplentes e inadimplentes se manteve praticamente a mesma.

## 6.2 Análise com base nas categorias

O que ficou claro na análise gráfica dos dados com base nas categorias escolaridade, estado civil e salario anual e que essas características não explicam maior ou menor índice de inandimplência. As proporções entre default e não default com relação a massa total se mantem praticamente iguais. Ou seja, não sinalizam nenhuma distorção que explique a inadimplência. 

## 6.3 Análise com base nas variáveis numéricas

Já com relação a análise dos gráficos das variáveis quantidade de transações em 12 meses e valor das transações em 12 meses é possível perceber que existe um comportamento anômalo para uma faixa específica de valor de transações e quantidade de transações.

Existe um forte incremento de clientes inadimplentes na faixa entre 35 e 45 transações anuais e na faixa média de valor de transações em R$ 2.500,00.

Quando sobrepomos as duas amostras no gráfico de relacionamento fica ainda mais claro a existência de uma relação entre o número de transações na faixa 35-45 com os valores médios de R$ 2.500,00, indicando uma atenção especial para esse grupo de clientes.

